# EthicML

## Runnning experiments on the Adult dataset

### Installation

First we need to install EthicML. Currently, the toolkit isn't on PyPi, but this will change soon.

For now, the toolkit has to be cloned, then installed as an editable package
```
cd <Location to clone to>
git clone https://github.com/predictive-analytics-lab/EthicML.git
cd EthicML
pip install --editable ./
```

(Obviously this notebook is within the package, so we can skip this step)

### Loading the data

EthicML includes some often used datasets from fairness literature.
First, we load one of these... in this example we load the UCI Adult dataset

In [1]:
import ethicml as em

data: em.DataTuple = em.adult().load()
assert (45222, 101) == data.x.shape
assert (45222, 1) == data.s.shape
assert (45222, 1) == data.y.shape

This loads the dataset as a DataTuple, which comprises $x$ (features), $s$ (sensitive attribute and $y$ (class label). Each member of the DataTuple is stored as a Pandas DataFrame.

By default, the Adult dataset uses the binary attribute `sex_Male` as the sensitive feature.

In [2]:
data.s.head()

,sex_Male
0,1
1,1
2,1
3,1
4,0


If we want to run experiments using race as the sensitive attribute we could change that manually, or, as this is a common task, EthicML can split the data for you.

In [3]:
data: em.DataTuple = em.adult(split="Race").load()
assert (45222, 98) == data.x.shape
assert (45222, 1) == data.s.shape
assert (45222, 1) == data.y.shape

In [4]:
data.s.head()

,race
0,4
1,4
2,4
3,2
4,4


However, we're going to be repeating some of the experiments from FairGP. In that paper they do experiments with race as the sensitive attribute, but the value is binary. The value of race is White or Not_White.

Fortunately, race has been one-hot-encoded so to replicate this we can just drop the features from the sensitive attribute that aren't `race_White`.

The Dataset class is really just a guide that tells EthicML how to read the underlying CSV. So to remove the other race attributes, we can just not include them in our list of sensitive attribute columns.

In [5]:
from dataclasses import replace

dataset = em.adult("Race")
dataset = replace(dataset, sens_attr_spec="race_White")
data = em.load_data(dataset)

In [6]:
data.s.head()

,race_White
0,1
1,1
2,1
3,0
4,1


### Evaluating some models

In [7]:
datasets = [dataset, em.toy()]
preprocess_models = [em.Upsampler()]
# inprocess_models = [em.Agarwal(), em.Kamishima(), em.LR(), em.SVM(kernel='linear'), em.Kamiran()]
inprocess_models = [em.LR(), em.SVM(kernel='linear'), em.Kamiran()]
postprocess_models = []
metrics = [em.Accuracy(), em.CV()]
per_sens_metrics = [em.Accuracy(), em.TPR(), em.ProbPos()]

In [8]:
test123 = em.evaluate_models(datasets, preprocess_models, inprocess_models, postprocess_models, metrics, per_sens_metrics, test_mode=False, repeats=2)

100%|██████████| 28/28 [00:13<00:00,  2.12it/s, model=Kamiran & Calders LR, dataset=Toy, transform=Upsample uniform, repeat=1]              


In [9]:
test123

Accuracy  \
dataset    transform        model                       split_id             
Adult Race no_transform     Logistic Regression, C=1.0  0         0.851410   
                            SVM                         0         0.851962   
                            Logistic Regression, C=1.0  1         0.841902   
                            SVM                         1         0.843449   
                            Logistic Regression (C=1.0) 0         0.851410   
                            SVM (linear)                0         0.851962   
                            Kamiran & Calders LR        0         0.850636   
                            Logistic Regression (C=1.0) 1         0.842676   
                            SVM (linear)                1         0.843449   
                            Kamiran & Calders LR        1         0.844002   
                            Logistic Regression (C=1.0) 0         0.851410   
                            SVM (linear)                0         0.851962   
                            Kamiran & Calders LR        0         0.850636   
                            Logistic Regression (C=1.0) 1         0.842676   
                            SVM (linear)                1         0.843449   
                            Kamiran & Calders LR        1         0.844002   
           Upsample uniform Logistic Regression (C=1.0) 0         0.850083   
                            SVM (linear)                0         0.850193   
                            Kamiran & Calders LR        0         0.850083   
                            Logistic Regression (C=1.0) 1         0.842565   
                            SVM (linear)                1         0.840575   
                            Kamiran & Calders LR        1         0.842123   
                            Logistic Regression (C=1.0) 0         0.850083   
                            SVM (linear)                0         0.850193   
                            Kamiran & Calders LR        0         0.850083   
                            Logistic Regression (C=1.0) 1         0.842565   
                            SVM (linear)                1         0.840575   
                            Kamiran & Calders LR        1         0.842123   
Toy        no_transform     Logistic Regression, C=1.0  0         0.885000   
                            SVM                         0         0.885000   
                            Logistic Regression, C=1.0  1         0.862500   
                            SVM                         1         0.860000   
                            Logistic Regression (C=1.0) 0         0.937500   
                            SVM (linear)                0         0.950000   
                            Kamiran & Calders LR        0         0.937500   
                            Logistic Regression (C=1.0) 1         1.000000   
                            SVM (linear)                1         0.987500   
                            Kamiran & Calders LR        1         1.000000   
                            Logistic Regression (C=1.0) 0         0.937500   
                            SVM (linear)                0         0.950000   
                            Kamiran & Calders LR        0         0.937500   
                            Logistic Regression (C=1.0) 1         1.000000   
                            SVM (linear)                1         0.987500   
                            Kamiran & Calders LR        1         1.000000   
           Upsample uniform Logistic Regression (C=1.0) 0         0.975000   
                            SVM (linear)                0         0.950000   
                            Kamiran & Calders LR        0         0.975000   
                            Logistic Regression (C=1.0) 1         1.000000   
                            SVM (linear)                1         1.000000   
                            Kamiran & Calders LR        1         1.000000   
          

In [10]:
test123.groupby(level=[0,1,2]).agg(['mean', 'std'])[['Accuracy', 'Accuracy_race_White_0/race_White_1', 'TPR_race_White_0/race_White_1', 'prob_pos_race_White_0/race_White_1']]

Accuracy            \
                                                             mean       std   
dataset    transform        model                                             
Adult Race Upsample uniform Kamiran & Calders LR         0.846103  0.004596   
                            Logistic Regression (C=1.0)  0.846324  0.004340   
                            SVM (linear)                 0.845384  0.005553   
           no_transform     Kamiran & Calders LR         0.847319  0.003830   
                            Logistic Regression (C=1.0)  0.847043  0.005043   
                            Logistic Regression, C=1.0   0.846656  0.006723   
                            SVM                          0.847706  0.006020   
                            SVM (linear)                 0.847706  0.004915   
Toy        Upsample uniform Kamiran & Calders LR         0.987500  0.014434   
                            Logistic Regression (C=1.0)  0.987500  0.014434   
                            SVM (linear)                 0.975000  0.028868   
           no_transform     Kamiran & Calders LR         0.968750  0.036084   
                            Logistic Regression (C=1.0)  0.968750  0.036084   
                            Logistic Regression, C=1.0   0.873750  0.015910   
                            SVM                          0.872500  0.017678   
                            SVM (linear)                 0.968750  0.021651   

                                                        Accuracy_race_White_0/race_White_1  \
                                                                                      mean   
dataset    transform        model                                                            
Adult Race Upsample uniform Kamiran & Calders LR                                  0.943158   
                            Logistic Regression (C=1.0)                           0.941924   
                            SVM (linear)                                          0.942148   
           no_transform     Kamiran & Calders LR                                  0.943171   
                            Logistic Regression (C=1.0)                           0.939846   
                            Logistic Regression, C=1.0                            0.942857   
                            SVM                                                   0.940723   
                            SVM (linear)                                          0.940723   
Toy        Upsample uniform Kamiran & Calders LR                                       NaN   
                            Logistic Regression (C=1.0)                                NaN   
                            SVM (linear)                                               NaN   
           no_transform     Kamiran & Calders LR                                       NaN   
                            Logistic Regression (C=1.0)                                NaN   
                            Logistic Regression, C=1.0                                 NaN   
                            SVM                                                        NaN   
                            SVM (linear)                                               NaN   

                                                                   \
                                                              std   
dataset    transform        model                                   
Adult Race Upsample uniform Kamiran & Calders LR         0.004163   
                            Logistic Regression (C=1.0)  0.002738   
                            SVM (linear)                 0.001691   
           no_transform     Kamiran & Calders LR         0.000039   
                            Logistic Regression (C=1.0)  0.000519   
                            Logistic Regression, C=1.0   0.002242   
                            SVM                          0.000971   
                            SVM (linear)                 0.000793   
Toy      

In [ ]:
datasets = [em.adult()]
preprocess_models = []
inprocess_models = [em.Agarwal(), em.Kamishima(), em.LR(), em.SVM()]
postprocess_models = []
metrics = [em.Accuracy(), em.CV()]
per_sens_metrics = [em.Accuracy(), em.TPR(), em.ProbPos()]
test123 = await em.evaluate_models_async(datasets, preprocess_models, inprocess_models, postprocess_models, metrics, per_sens_metrics, test_mode=False, repeats=10)

synchronous algorithms...


 65%|██████▌   | 13/20 [06:51<03:41, 31.66s/it, model=SVM, dataset=Adult Sex - Train (3)]                        

In [ ]:
test123.groupby(level=[0,1,2]).agg(['mean', 'std'])[['Accuracy', 'Accuracy_sex_Male_0/sex_Male_1', 'TPR_sex_Male_0/sex_Male_1', 'prob_pos_sex_Male_0/sex_Male_1']]
